# Extracting NDVI using Google Earth Engine




## Initializing GEE API 

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Enter verification code:  4/1AfgeXvt3a5cayVuAaRtPS5-wp50lyV_5MrAbEn8gTz4T6J6NqB34quwHeHo



Successfully saved authorization token.
Google Earth Engine has initialized successfully!


## Getting Data

In [2]:
#using ggemap python package for interactive mapping
import geemap

In [3]:
#Sentinel 2 cloud-free image in the Netherlands
collection = ('COPERNICUS/S2')
list_netherlands = list([(3.31, 50.80),(7.093, 50.80),(7.093, 53.52),(3.31, 53.52),(3.31, 50.80)])
area_netherlands = ee.Geometry.Polygon(list_netherlands)

In [4]:
#Gather images of the Netehrlands  

myCollection = ee.ImageCollection(collection) \
    .filterBounds(area_netherlands) \
    .filterDate('2017-01-01', '2017-07-01') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 20) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)

In [5]:
listOfImages = myCollection.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', len(listOfImages))

Number of images in the collection:  148


In [6]:
# Reduce the Img collection by mean.
img = myCollection.median()

In [7]:
# Compute Normalized Difference Vegetation Index over S2-L2 product.
# NDVI = (NIR - RED) / (NIR + RED), where
# RED is B4, 664.5 nm
# NIR is B8, 835.1 nm

# Calculate the NDVI manually: NDVI = (B8 - B4) / (B8 + B4)
nir = img.select('B8')
red = img.select('B4')
ndvi = nir.subtract(red).divide(nir.add(red))

# Display the result. 
ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']} 
Map = geemap.Map()
center = ee.Geometry.Point([5.29, 52.13]).getInfo()['coordinates']
Map.setCenter(center[0], center[1], 7)
Map.addLayer(ndvi, ndviParams, 'NDVI image #1')
Map

Map(center=[52.13, 5.29], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [8]:
bscale = ndvi.select('B8').projection().nominalScale() 
print('Band 8 scale: ', bscale.getInfo()) # ee.Number

Band 8 scale:  111319.49079327357


### Clip an image by region of interest

https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api#clip_an_image_by_a_region_of_interest

In [9]:
#Clip an image by region of interest

# Get a feature collection of administrative boundaries.
countries = ee.FeatureCollection('FAO/GAUL/2015/level0').select('ADM0_NAME')

# Filter the feature collection to subset Netherlands.
Netherlands = countries.filter(ee.Filter.eq('ADM0_NAME', 'Netherlands'))

# Clip the image by Netherlands.
ndvi_nl = ndvi.clip(Netherlands)

# Display the result. 
ndviParams = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']} 
Map = geemap.Map()
center = ee.Geometry.Point([5.29, 52.13]).getInfo()['coordinates']
Map.setCenter(center[0], center[1], 7)
Map.addLayer(ndvi_nl, ndviParams, 'NDVI image #1')
Map

Map(center=[52.13, 5.29], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [22]:
# Export to GEOTIFF file with the pixel resolution of 1km*1Km and with Dutch reference system (to be same and match with the block map)
task = ee.batch.Export.image.toDrive(image=ndvi,
                                     description='ndvi_Netherlands',
                                     scale=1000,
                                     region=area_netherlands,
                                     fileNamePrefix='ndvi_raster',
                                     crs='EPSG: 28992',
                                     fileFormat='GeoTIFF')
task.start()

In [23]:
task.status()

{'state': 'READY',
 'description': 'ndvi_Netherlands',
 'creation_timestamp_ms': 1667387842019,
 'update_timestamp_ms': 1667387842019,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'PDHRXVS4UP2J25DO24V2ZVWM',
 'name': 'projects/earthengine-legacy/operations/PDHRXVS4UP2J25DO24V2ZVWM'}